In [113]:
import pandas as pd
import requests

## Wczytywanie danych ELO drużyn piłkarskich i usuwanie zbędnych kolumn i wierszy.

In [7]:
elo = pd.read_csv("data/clubelo-club-rankings.csv")

In [8]:
elo = elo[elo['date'] >= '2024-08-14'] # data pierwszego meczu Mbappe dla Realu
elo = elo[['Club', 'Elo', 'date']]
elo.head()

,Club,Elo,date
275748,Man City,2050.554688,2024-08-14
275749,Real Madrid,1997.407959,2024-08-14
275750,Inter,1964.768311,2024-08-14
275751,Arsenal,1946.879395,2024-08-14
275752,Leverkusen,1925.011475,2024-08-14


## Pobranie z API danych o meczach Realu w sezonach 2024/25 i 2025/26.

In [12]:
# 2024/25

url = "http://api.football-data.org/v4/teams/86/matches?season=2024"
headers = {"X-Auth-Token": "KLUCZ_API"}

response = requests.get(url, headers=headers)
data_matches_2024 = response.json()    

df24 = pd.DataFrame(data_matches_2024['matches'])

# 2025/26

url = "http://api.football-data.org/v4/teams/86/matches"
headers = {"X-Auth-Token": "KLUCZ_API"}

response = requests.get(url, headers=headers)
data_matches_2025 = response.json()     

df25 = pd.DataFrame(data_matches_2025['matches'])

## Wyciąganie interesujących mnie danych.

In [29]:
# 2024/25

home24 = pd.DataFrame(df24['homeTeam'])

homeTeam_df = pd.json_normalize(home24['homeTeam'])
homeTeam_df.columns = ['homeTeam_' + col for col in homeTeam_df.columns]

home24 = home24.drop(columns=['homeTeam']).join(homeTeam_df)
home24 = home24[['homeTeam_id', 'homeTeam_name', 'homeTeam_shortName']]

away24 = pd.DataFrame(df24['awayTeam'])

awayTeam_df = pd.json_normalize(away24['awayTeam'])
awayTeam_df.columns = ['awayTeam_' + col for col in awayTeam_df.columns]

away24 = away24.drop(columns=['awayTeam']).join(awayTeam_df)
away24 = away24[['awayTeam_id', 'awayTeam_name', 'awayTeam_shortName']]

score24 = pd.DataFrame(df24['score'])

score_df = pd.json_normalize(score24['score'])
score_df.columns = [col for col in score_df.columns]

score24 = score24.drop(columns=['score']).join(score_df)

# Usunięcie z wyniku rzutów karnych
score24.loc[score24['duration'] == 'PENALTY_SHOOTOUT', ['fullTime.home', 'fullTime.away']] = [1, 0]

score24 = score24[['winner', 'fullTime.home', 'fullTime.away']]

date24 = df24[['utcDate']]

matches24 = pd.concat([home24, away24, score24, date24], axis=1)

# 2025/26

home25 = pd.DataFrame(df25['homeTeam'])

homeTeam_df = pd.json_normalize(home25['homeTeam'])
homeTeam_df.columns = ['homeTeam_' + col for col in homeTeam_df.columns]

home25 = home25.drop(columns=['homeTeam']).join(homeTeam_df)
home25 = home25[['homeTeam_id', 'homeTeam_name', 'homeTeam_shortName']]

away25 = pd.DataFrame(df25['awayTeam'])

awayTeam_df = pd.json_normalize(away25['awayTeam'])
awayTeam_df.columns = ['awayTeam_' + col for col in awayTeam_df.columns]

away25 = away25.drop(columns=['awayTeam']).join(awayTeam_df)
away25 = away25[['awayTeam_id', 'awayTeam_name', 'awayTeam_shortName']]

score25 = pd.DataFrame(df25['score'])

score_df = pd.json_normalize(score25['score'])
score_df.columns = [col for col in score_df.columns]

score25 = score25.drop(columns=['score']).join(score_df)

score25 = score25[['winner', 'fullTime.home', 'fullTime.away']]

date25 = df25[['utcDate']]

matches25 = pd.concat([home25, away25, score25, date25], axis=1)

#Usunięcie meczów, ktore się jeszcze nie odbyły
matches25 = matches25[matches25['winner'].isin(['HOME_TEAM', 'AWAY_TEAM', 'DRAW'])]

In [30]:
matches24.head()

,homeTeam_id,homeTeam_name,homeTeam_shortName,awayTeam_id,awayTeam_name,awayTeam_shortName,winner,fullTime.home,fullTime.away,utcDate
0,89,RCD Mallorca,Mallorca,86,Real Madrid CF,Real Madrid,DRAW,1,1,2024-08-18T19:30:00Z
1,86,Real Madrid CF,Real Madrid,250,Real Valladolid CF,Valladolid,HOME_TEAM,3,0,2024-08-25T15:00:00Z
2,275,UD Las Palmas,Las Palmas,86,Real Madrid CF,Real Madrid,DRAW,1,1,2024-08-29T19:30:00Z
3,86,Real Madrid CF,Real Madrid,90,Real Betis Balompié,Real Betis,HOME_TEAM,2,0,2024-09-01T19:30:00Z
4,92,Real Sociedad de Fútbol,Real Sociedad,86,Real Madrid CF,Real Madrid,AWAY_TEAM,0,2,2024-09-14T19:00:00Z


In [31]:
matches25.head()

,homeTeam_id,homeTeam_name,homeTeam_shortName,awayTeam_id,awayTeam_name,awayTeam_shortName,winner,fullTime.home,fullTime.away,utcDate
0,86,Real Madrid CF,Real Madrid,79,CA Osasuna,Osasuna,HOME_TEAM,1.0,0.0,2025-08-19T19:00:00Z
1,1048,Real Oviedo,Real Oviedo,86,Real Madrid CF,Real Madrid,AWAY_TEAM,0.0,3.0,2025-08-24T19:30:00Z
2,86,Real Madrid CF,Real Madrid,89,RCD Mallorca,Mallorca,HOME_TEAM,2.0,1.0,2025-08-30T19:30:00Z
3,92,Real Sociedad de Fútbol,Real Sociedad,86,Real Madrid CF,Real Madrid,AWAY_TEAM,1.0,2.0,2025-09-13T14:15:00Z
4,86,Real Madrid CF,Real Madrid,516,Olympique de Marseille,Marseille,HOME_TEAM,2.0,1.0,2025-09-16T19:00:00Z


## Tworzenie tabeli w czytelniejszej postaci.

In [32]:
def get_score(row):
    if row['winner'] == 'DRAW':
        return 'DRAW'
    elif ((row['winner'] == 'HOME_TEAM') and (row['homeTeam_name'] == 'Real Madrid CF')) \
        or ((row['winner'] == 'AWAY_TEAM') and (row['awayTeam_name'] == 'Real Madrid CF')):
        return 'WIN'
    else:
        return 'LOSS'

matches24_temp = matches24.copy()
matches25_temp = matches25.copy()

matches24 = pd.DataFrame({
    'Team_id': matches24_temp.apply(lambda row: row['awayTeam_id'] if row['homeTeam_name']=='Real Madrid CF' else row['homeTeam_id'], axis=1),
    'Team_name': matches24_temp.apply(lambda row: row['awayTeam_name'] if row['homeTeam_name']=='Real Madrid CF' else row['homeTeam_name'], axis=1),
    'Team_shortName': matches24_temp.apply(lambda row: row['awayTeam_shortName'] if row['homeTeam_name']=='Real Madrid CF' else row['homeTeam_shortName'], axis=1),
    'Score': matches24_temp.apply(get_score, axis=1),
    'goalsMadrid': matches24_temp.apply(lambda row: row['fullTime.home'] if row['homeTeam_name']=='Real Madrid CF' else row['fullTime.away'], axis=1),
    'goalsOpp': matches24_temp.apply(lambda row: row['fullTime.away'] if row['homeTeam_name']=='Real Madrid CF' else row['fullTime.home'], axis=1),
    'loc': matches24_temp.apply(lambda row: 'HOME' if row['homeTeam_name']=='Real Madrid CF' else 'AWAY', axis=1),
    'date': matches24_temp['utcDate']
})

matches25 = pd.DataFrame({
    'Team_id': matches25_temp.apply(lambda row: row['awayTeam_id'] if row['homeTeam_name']=='Real Madrid CF' else row['homeTeam_id'], axis=1),
    'Team_name': matches25_temp.apply(lambda row: row['awayTeam_name'] if row['homeTeam_name']=='Real Madrid CF' else row['homeTeam_name'], axis=1),
    'Team_shortName': matches25_temp.apply(lambda row: row['awayTeam_shortName'] if row['homeTeam_name']=='Real Madrid CF' else row['homeTeam_shortName'], axis=1),
    'Score': matches25_temp.apply(get_score, axis=1),
    'goalsMadrid': matches25_temp.apply(lambda row: row['fullTime.home'] if row['homeTeam_name']=='Real Madrid CF' else row['fullTime.away'], axis=1),
    'goalsOpp': matches25_temp.apply(lambda row: row['fullTime.away'] if row['homeTeam_name']=='Real Madrid CF' else row['fullTime.home'], axis=1),
    'loc': matches25_temp.apply(lambda row: 'HOME' if row['homeTeam_name']=='Real Madrid CF' else 'AWAY', axis=1),
    'date': matches25_temp['utcDate']
})

matches = pd.concat([matches24, matches25], ignore_index=True)

In [34]:
matches.head()

,Team_id,Team_name,Team_shortName,Score,goalsMadrid,goalsOpp,loc,date
0,89,RCD Mallorca,Mallorca,DRAW,1.0,1.0,AWAY,2024-08-18T19:30:00Z
1,250,Real Valladolid CF,Valladolid,WIN,3.0,0.0,HOME,2024-08-25T15:00:00Z
2,275,UD Las Palmas,Las Palmas,DRAW,1.0,1.0,AWAY,2024-08-29T19:30:00Z
3,90,Real Betis Balompié,Real Betis,WIN,2.0,0.0,HOME,2024-09-01T19:30:00Z
4,92,Real Sociedad de Fútbol,Real Sociedad,WIN,2.0,0.0,AWAY,2024-09-14T19:00:00Z


## Zmiana odpowiednich wartości w kolumnie Team_shortName, żeby zgadzały się z tabelą ELO.

In [56]:
name_map = {
    'Real Betis': 'Betis',
    'Real Sociedad': 'Sociedad',
    'Alavés': 'Alaves',
    'Atleti': 'Atletico',
    'Barça': 'Barcelona',
    'Leganés': 'Leganes',
    'Athletic': 'Bilbao',
    'Sevilla FC': 'Sevilla',
    'RB Salzburg': 'Salzburg',
    'Real Oviedo' : 'Oviedo',
    'FK Kairat' : 'Kairat'
}

In [57]:
matches['Team_shortName'] = matches['Team_shortName'].replace(name_map)

## Braki danych i ręczne uzupełnienie informacji

Po połączeniu dostępnych źródeł danych okazało się, że:
- baza meczowa pobrana z API nie zawierała pełnych informacji o bramkach i asystach zawodników Mbappé i Viníciusa,
- brakowało danych dla około 20 meczów.

Ze względu na brak alternatywnego, kompletnego darmowego źródła danych w formacie API, zdecydowałem się na ręczne uzupełnienie brakujących informacji na podstawie publicznie dostępnych statystyk meczowych.

W tym celu:
1. zapisano tabelę do pliku xlsx,
2. brakujące dane zostały uzupełnione ręcznie w arkuszu Excel,
3. poprawiony plik został ponownie wczytany do dalszej analizy.

In [35]:
matches.to_excel('matches.xlsx', index=False)

In [102]:
matches = pd.read_csv("matches.csv", sep = ',')

In [104]:
matches = matches.sort_values(by='date')

## Mergowanie tabeli z meczami z tabelą z ELO, w celu uzyskania ELO w każdym meczu.

In [105]:
# Upewnienie się, że formaty dat są takie same
elo['date'] = pd.to_datetime(elo['date'])
matches['date'] = pd.to_datetime(matches['date'])

matches_elo = matches.merge(
    elo[elo['Club'] == 'Real Madrid'].rename(columns={'Elo':'eloMadrid'}),
    left_on='date', right_on='date', how='left'
)

matches_elo = matches_elo.merge(
    elo.rename(columns={'Club':'Team_shortName','Elo':'eloOpp'}),
    left_on=['Team_shortName','date'],
    right_on=['Team_shortName','date'],
    how='left'
)

matches_elo = matches_elo.drop(columns = ['Club'])

In [106]:
matches_elo.head()

,Unnamed: 0,Team_id,Team_name,Team_shortName,Score,goalsMadrid,goalsOpp,loc,date,Mbappe_play,Vini_play,MbappeG,MbappeA,ViniG,ViniA,eloMadrid,eloOpp
0,77,0,Atalanta BC,Atalanta,WIN,2,0,NEUTRAL,2024-08-14,YES,YES,1,0,0,1,1997.407959,1884.456909
1,0,89,RCD Mallorca,Mallorca,DRAW,1,1,AWAY,2024-08-18,YES,YES,0,0,0,1,1997.407959,1642.396118
2,1,250,Real Valladolid CF,Valladolid,WIN,3,0,HOME,2024-08-25,YES,YES,0,0,0,0,1990.887939,1600.702026
3,2,275,UD Las Palmas,Las Palmas,DRAW,1,1,AWAY,2024-08-29,YES,YES,0,0,1,0,1992.368896,1582.391235
4,3,90,Real Betis BalompiĂ©,Betis,WIN,2,0,HOME,2024-09-01,YES,YES,2,0,0,1,1985.119385,1711.915771


# Niepełne dane ELO.
- Dla meczy, które były rozgrywane w dniu, dla których nie ma danych, wartości zostaną uzupełnione ręcznie o dane z najbliższego wcześniejszego.
- W tabeli jest 5 meczy z drużynami należącymi do rozgrywek, które nie są uwzględnione w rankingu ELO, więc zostaną one usunięte.

In [107]:
matches_elo[matches_elo['eloOpp'].isna()]

,Unnamed: 0,Team_id,Team_name,Team_shortName,Score,goalsMadrid,goalsOpp,loc,date,Mbappe_play,Vini_play,MbappeG,MbappeA,ViniG,ViniA,eloMadrid,eloOpp
24,78,0,Pachuca,Pachuca,WIN,3,0,NEUTRAL,2024-12-18,YES,YES,1,1,1,1,1947.474365,NaN
25,23,559,Sevilla FC,Sevilla,WIN,4,2,HOME,2024-12-22,YES,NO,1,1,0,0,NaN,NaN
26,24,95,Valencia CF,Valencia,WIN,2,1,AWAY,2025-01-03,YES,YES,0,0,0,0,NaN,NaN
27,79,0,Deportiva Minera,Minera,WIN,5,0,AWAY,2025-01-06,YES,YES,0,0,0,0,NaN,NaN
28,80,0,RCD Mallorca,Mallorca,WIN,3,0,NEUTRAL,2025-01-09,YES,YES,0,0,0,0,NaN,NaN
62,89,0,Al-Hilal,Al-Hilal,DRAW,1,1,NEUTRAL,2025-06-18,NO,YES,0,0,0,0,1936.127319,NaN
63,90,0,Pachuca,Pachuca,WIN,3,1,NEUTRAL,2025-06-22,NO,YES,0,0,0,0,1936.127319,NaN
78,62,82,Getafe CF,Getafe,WIN,1,0,AWAY,2025-10-19,YES,YES,1,0,0,0,NaN,NaN
91,87,0,CF Talavera,Talavera,WIN,3,2,AWAY,2025-12-17,YES,NO,2,0,0,0,1907.254517,NaN
94,88,0,Club AtlĂ©tico de Madrid,Atletico,WIN,2,1,NEUTRAL,2026-01-08,NO,YES,0,0,0,0,NaN,NaN


In [108]:
matches_elo.loc[matches_elo['date'] == '2024-12-22', 'eloMadrid'] = elo[(elo['date'] == '2024-12-19') & (elo['Club'] == 'Real Madrid')]['Elo'].values[0]
matches_elo.loc[matches_elo['date'] == '2024-12-22', 'eloOpp'] = elo[(elo['date'] == '2024-12-19') & (elo['Club'] == 'Sevilla')]['Elo'].values[0]
matches_elo.loc[matches_elo['date'] == '2025-01-03', 'eloMadrid'] = elo[(elo['date'] == '2024-12-19') & (elo['Club'] == 'Real Madrid')]['Elo'].values[0]
matches_elo.loc[matches_elo['date'] == '2025-01-03', 'eloOpp'] = elo[(elo['date'] == '2024-12-19') & (elo['Club'] == 'Valencia')]['Elo'].values[0]
matches_elo.loc[matches_elo['date'] == '2025-10-19', 'eloMadrid'] = elo[(elo['date'] == '2025-10-18') & (elo['Club'] == 'Real Madrid')]['Elo'].values[0]
matches_elo.loc[matches_elo['date'] == '2025-10-19', 'eloOpp'] = elo[(elo['date'] == '2025-10-18') & (elo['Club'] == 'Getafe')]['Elo'].values[0]

matches_elo.loc[matches_elo['date'] == '2025-01-06', 'eloMadrid'] = elo[(elo['date'] == '2024-12-19') & (elo['Club'] == 'Real Madrid')]['Elo'].values[0]
matches_elo.loc[matches_elo['date'] == '2025-01-09', 'eloMadrid'] = elo[(elo['date'] == '2024-12-19') & (elo['Club'] == 'Real Madrid')]['Elo'].values[0]
matches_elo.loc[matches_elo['date'] == '2025-01-09', 'eloOpp'] = elo[(elo['date'] == '2024-12-19') & (elo['Club'] == 'Mallorca')]['Elo'].values[0]
matches_elo.loc[matches_elo['date'] == '2024-12-22', 'eloOpp'] = elo[(elo['date'] == '2024-12-19') & (elo['Club'] == 'Sevilla')]['Elo'].values[0]
matches_elo.loc[matches_elo['date'] == '2026-01-08', 'eloMadrid'] = elo[(elo['date'] == '2026-01-05') & (elo['Club'] == 'Real Madrid')]['Elo'].values[0]
matches_elo.loc[matches_elo['date'] == '2026-01-08', 'eloOpp'] = elo[(elo['date'] == '2026-01-05') & (elo['Club'] == 'Atletico')]['Elo'].values[0]

In [109]:
matches_elo[matches_elo['eloOpp'].isna()]

,Unnamed: 0,Team_id,Team_name,Team_shortName,Score,goalsMadrid,goalsOpp,loc,date,Mbappe_play,Vini_play,MbappeG,MbappeA,ViniG,ViniA,eloMadrid,eloOpp
24,78,0,Pachuca,Pachuca,WIN,3,0,NEUTRAL,2024-12-18,YES,YES,1,1,1,1,1947.474365,NaN
27,79,0,Deportiva Minera,Minera,WIN,5,0,AWAY,2025-01-06,YES,YES,0,0,0,0,1947.474365,NaN
62,89,0,Al-Hilal,Al-Hilal,DRAW,1,1,NEUTRAL,2025-06-18,NO,YES,0,0,0,0,1936.127319,NaN
63,90,0,Pachuca,Pachuca,WIN,3,1,NEUTRAL,2025-06-22,NO,YES,0,0,0,0,1936.127319,NaN
91,87,0,CF Talavera,Talavera,WIN,3,2,AWAY,2025-12-17,YES,NO,2,0,0,0,1907.254517,NaN


In [110]:
matches_elo = matches_elo[matches_elo['eloOpp'].notna()]

In [111]:
matches_elo[matches_elo['eloOpp'].isna()]

,Unnamed: 0,Team_id,Team_name,Team_shortName,Score,goalsMadrid,goalsOpp,loc,date,Mbappe_play,Vini_play,MbappeG,MbappeA,ViniG,ViniA,eloMadrid,eloOpp


# Zapisanie finalnej tabeli

In [112]:
matches_elo.to_csv("matches_elo.csv", index=False)